In [ ]:
import pandas as pd
import numpy as np
import re
import glob
import seaborn as sns
import matplotlib.pyplot as plt
from Bresenheim import *
import pickle
from itertools import product
from scipy import ndimage
from numpy.linalg import norm
sns.set_style('darkgrid')
sns.set(font_scale = 1.4)

In [ ]:
# load tracks : 
files = glob.glob('../data/real_cells/exp*.txt')
filename = files[0]
df = pd.read_csv(files[0], sep="\t", names=['cell_id', 'time_id', 'time', 'x', 'y', 'z'])   
df['experiment'] = filename[19:28]
df['cell_type'] = filename[29:32]
df['movie'] = filename[38:39]

In [ ]:
df.head()
# df[['x','y','z']]

In [ ]:
df_list = [df[df.cell_id == i] for i in df.cell_id.unique()]

In [ ]:
time_hist = [len(data.time.unique()) for data in df_list]
sns.distplot(time_hist)

In [ ]:
len([t for t in time_hist if t > 80])

In [ ]:
### extract tracks from all files
counter = 0
for f in files:
    print(f)
    df = pd.read_csv(f, sep="\t", names=['cell_id', 'time_id', 'time', 'x', 'y', 'z'])   
    df['experiment'] = f[19:28]
    df['cell_type'] = f[29:32]
    df['movie'] = f[38:39]
    df = df[df.cell_type == 'cd8']
    df_list = [df[df.cell_id == i] for i in df.cell_id.unique()]
    for data in df_list:
        if len(data.time.unique()) > 80:
            track = data[['x','y','z']].to_numpy()
            name = '../data/real_cells/Mlab_track' + str(counter) + '.txt'
            np.savetxt(name,track)
            counter += 1

In [ ]:
def auto_cor(cell_track): 
    averages = []
    for dt in range(0,60):
        # angles per dt: 
        cosines = []
        for i in range(len(cell_track) - 2 - dt):
            point1 = cell_track[i]
            point2 = cell_track[i + 1]
            point3 = cell_track[i + dt]
            point4 = cell_track[i + dt + 1]
            v1 = point2 - point1
            v2 = point4 - point3
            #cos = np.clip(np.dot(v1,v2)/(norm(v1) * norm(v2)),0,1)
            cos = np.dot(v1,v2)/(norm(v1) * norm(v2))
            #cos = abs(np.dot(v1,v2)/(norm(v1) * norm(v2)))
            cosines.append(cos)
        #if len(cosines) > 100: #minimum 100 mcs with actual displacement
        averages.append(np.average(cosines))
    return averages

In [ ]:
track_files = glob.glob('../data/real_cells/Mlab_track*.txt')
autocorrelations = []
tracks = []
for f in track_files:
    track = np.loadtxt(f)
    AC = auto_cor(track)
    if np.nan not in AC:
        tracks.append(track)
        autocorrelations.append(AC)

In [ ]:
ac_df = pd.DataFrame(autocorrelations)
ac_df = ac_df.dropna()
ac_df.head()

In [ ]:
%matplotlib notebook
plt.figure(figsize = (10,10))
plt.plot(ac_df.T.to_numpy()[1:],alpha = 0.1)
plt.plot(np.average(ac_df.T.to_numpy()[1:],axis = 1))
plt.ylabel('Autocorrelation')
plt.xlabel('delta T')
plt.title('average autocorrelation of MotilityLab data')
plt.show

In [ ]:
np.average(ac_df.T.to_numpy(),axis = 1)

In [ ]:
%matplotlib notebook
plt.imshow(autocorrelations)
plt.colorbar()
plt.show()

In [ ]:
%matplotlib notebook
#autocorrelations = np.array(autocorrelations).T
#print(autocorrelations[0,:])
##plt.plot(range(len(autocorrelations) - 1),autocorrelations)
#plt.plot(autocorrelations[1:],alpha = .2)
plt.plot(np.average(autocorrelations[1:]))
plt.show()
print(np.average(autocorrelations[1:]))
#print(len(tracks),len(autocorrelations))

In [ ]:
from CPM_helpers1 import plot_celltrack
%matplotlib notebook

cell_track = tracks[68]

# plot path of center of mass  :
x = [c[0] for c in cell_track]
y = [c[1] for c in cell_track]
z = [c[2] for c in cell_track]

#fig = plt.figure()
ax = plt.axes(projection='3d')#projection='3d'
ax.plot3D(x,y,z)
ax.scatter3D(x[0],y[0],z[0],label = 'start')
ax.scatter3D(x[-1],y[-1],z[-1],label = 'end')
ax.legend()
plt.show()